In [1]:
         #ANÁLISIS DE DATOS DE E-COMMERCE - PROYECTO 

# Importamos las bibliotecas necesarias
import pandas as pd      
import numpy as np       
import matplotlib.pyplot as plt  
import seaborn as sns    
import os                
from datetime import datetime  
from matplotlib.ticker import FuncFormatter  

# Cargamos los 
clientes = pd.read_csv('customers_dataset.csv')
ubicaciones = pd.read_csv('geolocation_dataset.csv') 
pagos = pd.read_csv('order_payments_dataset.csv')
productos = pd.read_csv('products_dataset.csv')
vendedores = pd.read_csv('sellers_dataset.csv')
ordenes = pd.read_csv('orders_dataset.csv')
items_ordenes = pd.read_csv('order_items_dataset.csv')
reviews = pd.read_csv('order_reviews_dataset.csv')
traduccion_categorias = pd.read_csv('product_category_name_translation.csv')

# LIMPIEZA Y PREPARACIÓN DE DATOS (DATA WRANGLING)

print("Iniciando proceso de limpieza y preparación de datos...")

# 1. Verificación de datos faltantes
print("\n--- Verificando valores faltantes ---")
print("Clientes - valores faltantes:", clientes.isnull().sum().sum())
print("Ubicaciones - valores faltantes:", ubicaciones.isnull().sum().sum())
print("Pagos - valores faltantes:", pagos.isnull().sum().sum())
print("Productos - valores faltantes:", productos.isnull().sum().sum())
print("Vendedores - valores faltantes:", vendedores.isnull().sum().sum())
print("Ordenes - valores faltantes:", ordenes.isnull().sum().sum())
print("Items ordenes - valores faltantes:", items_ordenes.isnull().sum().sum())
print("Reviews - valores faltantes:", reviews.isnull().sum().sum())

# 2. Limpieza de datos - Eliminamos duplicados
clientes = clientes.drop_duplicates()
ubicaciones = ubicaciones.drop_duplicates()
pagos = pagos.drop_duplicates()
productos = productos.drop_duplicates()
vendedores = vendedores.drop_duplicates()
ordenes = ordenes.drop_duplicates()
items_ordenes = items_ordenes.drop_duplicates()
reviews = reviews.drop_duplicates()

# 3. Convertimos fechas a formato datetime
# Convertimos las fechas en la tabla de órdenes
ordenes['order_purchase_timestamp'] = pd.to_datetime(ordenes['order_purchase_timestamp'])
ordenes['order_approved_at'] = pd.to_datetime(ordenes['order_approved_at'])
ordenes['order_delivered_carrier_date'] = pd.to_datetime(ordenes['order_delivered_carrier_date'])
ordenes['order_delivered_customer_date'] = pd.to_datetime(ordenes['order_delivered_customer_date'])
ordenes['order_estimated_delivery_date'] = pd.to_datetime(ordenes['order_estimated_delivery_date'])

# Convertimos las fechas en la tabla de reviews
reviews['review_creation_date'] = pd.to_datetime(reviews['review_creation_date'])
reviews['review_answer_timestamp'] = pd.to_datetime(reviews['review_answer_timestamp'])

# 4. Identificamos valores extremos en precios
precio_promedio = items_ordenes['price'].mean()
precio_desviacion = items_ordenes['price'].std()
precio_minimo = items_ordenes['price'].min()
precio_maximo = items_ordenes['price'].max()

print("\n--- Análisis de precios ---")
print("Precio promedio:", precio_promedio)
print("Precio mínimo:", precio_minimo)
print("Precio máximo:", precio_maximo)

# 5. Traducción de categorías de productos
productos = productos.merge(traduccion_categorias, on='product_category_name', how='left')

# 6. Creamos columnas nuevas útiles
# Añadimos mes y año a las órdenes
ordenes['mes_compra'] = ordenes['order_purchase_timestamp'].dt.month
ordenes['año_compra'] = ordenes['order_purchase_timestamp'].dt.year

# Calculamos tiempo de entrega en días
ordenes['tiempo_entrega'] = (ordenes['order_delivered_customer_date'] - ordenes['order_purchase_timestamp']).dt.days

# 7. Mostramos información después de la limpieza
print("\n--- Resumen de datos después de la limpieza ---")
print("Clientes:", clientes.shape[0], "filas")
print("Ubicaciones:", ubicaciones.shape[0], "filas")
print("Pagos:", pagos.shape[0], "filas")
print("Productos:", productos.shape[0], "filas")
print("Vendedores:", vendedores.shape[0], "filas")
print("Ordenes:", ordenes.shape[0], "filas")
print("Items ordenes:", items_ordenes.shape[0], "filas")
print("Reviews:", reviews.shape[0], "filas")

print("\nProceso de limpieza y preparación de datos completado.")

# RFM ANALYSIS

# Calculamos la fecha más reciente en el dataset
fecha_maxima = ordenes['order_purchase_timestamp'].max()

# Unimos las tablas necesarias para el análisis RFM
df_rfm = ordenes.merge(items_ordenes, on='order_id')

# Calculamos los componentes del RFM por cliente
rfm_data = df_rfm.groupby('customer_id').agg({
    'order_purchase_timestamp': lambda x: (fecha_maxima - x.max()).days,  # Recency
    'order_id': 'nunique',  # Frequency
    'price': 'sum'  # Monetary
}).rename(columns={
    'order_purchase_timestamp': 'recency',
    'order_id': 'frequency',
    'price': 'monetary'
})

# Verificamos los resultados del cálculo RFM
print("Análisis RFM - Primeras filas:")
print(rfm_data.head())

# Estadísticas descriptivas del RFM
print("\nEstadísticas descriptivas del RFM:")
print(rfm_data.describe())

# Segmentación de clientes basada en RFM
# Dividimos cada métrica en 5 segmentos (quintiles)
quintiles = rfm_data.rank(pct=True).applymap(lambda x: int(x * 5) + 1)

# Invertimos la recencia (menor recencia = mejor puntuación)
quintiles['recency'] = 6 - quintiles['recency']

# Creamos una puntuación RFM combinada
rfm_data['rfm_score'] = quintiles['recency'] * 100 + quintiles['frequency'] * 10 + quintiles['monetary']

# Clasificamos a los clientes según su puntuación
def clasificar_cliente(row):
    if row['rfm_score'] >= 432:
        return 'Campeones'
    elif row['rfm_score'] >= 332:
        return 'Leales'
    elif row['rfm_score'] >= 232:
        return 'Potenciales'
    elif row['rfm_score'] >= 132:
        return 'En riesgo'
    else:
        return 'Necesitan atención'

rfm_data['segmento'] = rfm_data.apply(clasificar_cliente, axis=1)

# Visualizamos la distribución de segmentos
plt.figure(figsize=(10, 6))
sns.countplot(x='segmento', data=rfm_data, order=rfm_data['segmento'].value_counts().index)
plt.title('Distribución de Clientes por Segmento RFM')
plt.xlabel('Segmento')
plt.ylabel('Número de Clientes')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('segmentos_rfm.png')
plt.close()

print("\nDistribución de clientes por segmento:")
print(rfm_data['segmento'].value_counts())
# Análisis más profundo de los segmentos

# Estadísticas descriptivas por segmento
print("\nEstadísticas por segmento:")
segment_stats = rfm_data.groupby('segmento').agg({
    'recency': ['mean', 'min', 'max'],
    'frequency': ['mean', 'min', 'max'],
    'monetary': ['mean', 'min', 'max']
})
print(segment_stats)

# Visualización de las métricas RFM por segmento
plt.figure(figsize=(15, 10))

# Recencia promedio por segmento
plt.subplot(3, 1, 1)
sns.barplot(x='segmento', y='recency', data=rfm_data, estimator=np.mean, 
            order=['Campeones', 'Leales', 'Potenciales', 'En riesgo', 'Necesitan atención'])
plt.title('Recencia Promedio por Segmento')
plt.ylabel('Días desde última compra')
plt.xticks(rotation=45)

# Frecuencia promedio por segmento
plt.subplot(3, 1, 2)
sns.barplot(x='segmento', y='frequency', data=rfm_data, estimator=np.mean,
            order=['Campeones', 'Leales', 'Potenciales', 'En riesgo', 'Necesitan atención'])
plt.title('Frecuencia Promedio por Segmento')
plt.ylabel('Número de compras')
plt.xticks(rotation=45)

# Valor monetario promedio por segmento
plt.subplot(3, 1, 3)
sns.barplot(x='segmento', y='monetary', data=rfm_data, estimator=np.mean,
            order=['Campeones', 'Leales', 'Potenciales', 'En riesgo', 'Necesitan atención'])
plt.title('Valor Monetario Promedio por Segmento')
plt.ylabel('Valor promedio ($)')
plt.xticks(rotation=45)

plt.tight_layout()
plt.savefig('metricas_por_segmento.png')
plt.close()

# Recomendaciones de marketing basadas en segmentos
print("\nRecomendaciones de marketing por segmento:")
print("1. Campeones: Programas de fidelización, convertirlos en embajadores de marca")
print("2. Leales: Venta cruzada, programas de referidos")
print("3. Potenciales: Ofertas personalizadas, incentivos para aumentar frecuencia")
print("4. En riesgo: Campañas de reactivación, descuentos especiales")
print("5. Necesitan atención: Recordatorios, ofertas de bienvenida nuevamente")

# Exportamos los resultados para uso en campañas de marketing
rfm_data.to_csv('segmentacion_clientes_rfm.csv', index=True)
print("\nResultados exportados a 'segmentacion_clientes_rfm.csv'")


   
    


ModuleNotFoundError: No module named 'seaborn'